* Recap of Software Development Life Cycle
* Setup Git Repo for CI/CD Demo
* Review Virtual Machine to deploy the application
* Review Manual Deployment Process
* Getting Started with GitHub Actions
* Overview of GitHub Repository Variables
* Create GitHub Repository Secrets
* Validate GitHub Action and VM Integration
* Overview of Git Branching
* Overview of Git Pull Requests
* Develop Shell Script to Deploy using GitHub Actions
* Validate CI/CD Pipeline
* Exercise and Solution

* Recap of Software Development Life Cycle

SDLC have different phases.
* Analysis
* Design
* Development
* Test
* Production

We use different servers for development, test and production. Once the code is developed, only reviewed and validated/tested code will go into test and production environments.

* Setup Git Repo for CI/CD Demo

Here are the steps invovled in setting up git repo.
* Go to github and create a new repository by name cicd-demo.
* Create a Python file by name `hw.py` with below code.
```python
print('Hello World')
```
* Make sure to clone the repo to PC or Mac for local development.

* Review Virtual Machine to deploy the application

Make sure to login via SSH into the virtual machine created. You can also validate whether Python is installed or not on the VM.

* Review Manual Deployment Process

Here are the steps that are involved in manual deployment process.
* Login into VM using SSH where application needs to be deployed.
* Clone git repository and get into the directory
* Run the application using `python app.py`

* Getting Started with GitHub Actions

Here are the steps to get started with GitHub Actions.
* Go to GitHub repository and then navigate to Actions.
* Click on "set up a workflow yourself"
```yml
name: CICD Demo

on:
  push:
    branches: [ "main" ]
  pull_request:
    branches: [ "main" ]

permissions:
  contents: read

jobs:
  build:

    runs-on: ubuntu-latest

    steps:
    - uses: actions/checkout@v3
    - name: Hello World
      run: |
        echo `hostname`
        echo "Hello World"
```

* Overview of GitHub Repository Variables

Create Variable by name `FOO` with value `bar`.

```yml
name: CICD Demo

on:
  push:
    branches: [ "main" ]
  pull_request:
    branches: [ "main" ]

permissions:
  contents: read

jobs:
  build:

    runs-on: ubuntu-latest

    steps:
    - uses: actions/checkout@v3
    - name: Hello World
      run: |
        echo ${{ vars.FOO }}
        echo `hostname`
        echo "Hello World"
```

* Create GitHub Repository Secrets

Here are the details to create GitHub Repository Secrets. Secrets can be created by going to Settings of a specific repository.
|Secret Name|Description|
|---|---|
|APP_HOST|IP Address or DNS Alias of VM|
|APP_SSH_USER|Username to connect to VM|
|APP_SSH_SECRET_KEY|Private key for password less login|

As the VM is created using GCP, the content of `google_compute_engine` is used for `APP_SSH_SECRET_KEY`.

* Validate GitHub Action and VM Integration

```yml
name: CICD Demo

on:
  push:
    branches: [ "main" ]
  pull_request:
    branches: [ "main" ]

permissions:
  contents: read

jobs:
  build:

    runs-on: ubuntu-latest

    steps:
    - uses: actions/checkout@v3
    - name: Validate GitHub Action and VM Integration
      env:
        APP_HOST: ${{ secrets.APP_HOST }}
        APP_SSH_USER: ${{ secrets.APP_SSH_USER }}
        APP_SSH_SECRET_KEY: ${{ secrets.APP_SSH_SECRET_KEY }}
      run: |
        mkdir -p /tmp/keys && chmod 700 /tmp/keys
        echo "$APP_SSH_SECRET_KEY" > /tmp/keys/private_key && chmod 600 /tmp/keys/private_key
        ssh -o StrictHostKeyChecking=no -i /tmp/keys/private_key ${APP_SSH_USER}@${APP_HOST} '
          echo `hostname`
        '
```

* Overview of Git Branching

Here are the steps to create a branch.
* Checkout a new branch
* Make changes to `hw.py`
* Add, Commit and Push the changes in the new branch to Git repository.

* Overview of Git Pull Requests

Once the changes in the branch are pushed to Git repository, a PR can be created. Once PR is approved and changes in the new branch are merged into main branch, the GitHub action will be triggered.

* Develop Shell Script to Deploy using GitHub Actions
```shell
python3 -m venv cd-venv
source cd-venv/bin/activate
pip install flask
sudo systemctl restart nginx
```

* Validate CI/CD Pipeline

```yml
name: CICD Demo

on:
  push:
    branches: [ "main" ]
  pull_request:
    branches: [ "main" ]

permissions:
  contents: read

jobs:
  build:

    runs-on: ubuntu-latest

    steps:
    - uses: actions/checkout@v3
    - name: Deploy and Build
      env:
        APP_HOST: ${{ secrets.APP_HOST }}
        APP_SSH_USER: ${{ secrets.APP_SSH_USER }}
        APP_SSH_SECRET_KEY: ${{ secrets.APP_SSH_SECRET_KEY }}
      run: |
        echo `pwd`
        mkdir -p /tmp/keys && chmod 700 /tmp/keys
        echo "$APP_SSH_SECRET_KEY" > /tmp/keys/private_key && chmod 600 /tmp/keys/private_key
        ssh -o StrictHostKeyChecking=no -i /tmp/keys/private_key ${APP_SSH_USER}@${APP_HOST} '
          mkdir -p cicddemo
        '
        scp -i /tmp/keys/private_key -r * ${APP_SSH_USER}@${APP_HOST}:cicddemo
        ssh -i /tmp/keys/private_key ${APP_SSH_USER}@${APP_HOST} '
          source cicddemo/setup.sh
        '
```

* Exercise - CI/CD using GitHub

1. Create Git Repository for `sales-app`.
2. Add Secrets to deploy the application via GitHub Actions
3. Create GitHub Action by name `sales-app.yml`
4. Update `nginx.conf` with below details.
```text

```
5. Add below shell script to the base folder of the repository.
```shell
python3 -m venv sa-venv
source sa-venv/bin/activate
pip install -r requirements.txt
gunicorn -w 2 --threads 2 -b 0.0.0.0:5000 app:app
sudo systemctl restart nginx
```
6. Validate whether GitHub Action is run successfully or not.